# RAG with Azure AI Search

<img src="./architecture.png">

<img src="vector_embeddings.svg" >


### Installing Packages

In [3]:
%pip install openai
%pip install python-dotenv
%pip install requests




[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)
Using cached python_dotenv-1.0.1-py3-none-any.whl (19 kB)

[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


### Loading variables from the .env file

In [4]:
from openai import AzureOpenAI
import os
from dotenv import load_dotenv
load_dotenv()

service_endpoint = os.getenv("AZURE_SEARCH_SERVICE_ENDPOINT")
index_name = os.getenv("AZURE_SEARCH_INDEX_NAME")
key = os.getenv("AZURE_SEARCH_API_KEY")

### Creating an Azure OpenAI Client

In [5]:
from openai import AzureOpenAI

azure_openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")  
azure_openai_key = os.getenv("AZURE_OPENAI_KEY")

azure_openai_client = AzureOpenAI(
    api_key=azure_openai_key,
    api_version="2024-02-15-preview",
    azure_endpoint=azure_openai_endpoint
)

### Creating Embedding Generator Function

In [6]:
def generate_embeddings(client, text):
    embedding_model = os.getenv("EMBEDDING_ENGINE")
    
    response = client.embeddings.create(
        input=text,
        model = embedding_model
    )
    
    embeddings=response.model_dump()
    return embeddings['data'][0]['embedding']
    

In [21]:
user_query = "Me informe dados sobre o curso técnico de soldagem?"
vectorised_user_query = generate_embeddings(azure_openai_client, user_query)
print(vectorised_user_query)


[-0.022223221, -0.0070621325, 0.003302835, -0.033517417, -0.033778254, 0.028691951, -0.022888353, -0.021545047, -0.017293422, -0.014971982, 0.014932857, 0.008457605, 0.027779026, -0.016536998, 0.013335236, -0.0034169506, 0.010550813, 0.011887597, 0.007538158, -0.0159762, -0.012018016, -0.0047798185, 0.017019544, 0.0003959408, -0.0004915127, -0.0054579917, 0.004261407, -0.020019157, 0.00042793393, -0.029683128, 0.022197137, -0.021388546, -0.012520124, -0.016132701, -0.016536998, 0.0030843848, 0.0069643194, 0.0096313665, 0.012898336, 0.0040494776, 0.0177238, 0.0009691685, 0.02330569, -0.023266565, -0.0117245745, 0.0010588309, -0.03432601, 0.013041796, -0.041812, 0.010003057, 0.011665886, 0.004616796, -0.02381432, -0.031065559, -0.008842338, 0.01661525, -0.005109124, 0.01632833, 0.014046014, -0.020671247, 0.0064100428, -0.017945511, -0.02918754, -0.002526848, -0.0039679664, 0.016641332, -0.010466041, 0.027153019, -0.024505535, 0.015493654, -0.0020019158, 0.040768653, 0.009637888, -0.01772

In [22]:
context=[]

### Sending API call to the Search Index

In [23]:
import requests
import json


url = f"{service_endpoint}/indexes/{index_name}/docs/search?api-version=2023-11-01"
    
headers = {
        "Content-Type": "application/json",
        "api-key": key
    }
    
body =   {
        "count": True,
        "select": "chunk",
        "vectorQueries": [
            {
                "vector": vectorised_user_query,
                "k": 3,
                "fields": "text_vector",
                "kind": "vector"
            }
        ]
    }
    
response = requests.post(url, headers=headers, data=json.dumps(body))
documents = response.json()['value']

for doc in documents:
    context.append(dict(
        {
            "chunk": doc['chunk'],
            "score": doc['@search.score']
            
        }
    ))
    
for doc in context:
    print(doc)



{'chunk': 'média anual entre 2018 e 2023, na área \nde Metalmecânica – Soldagem.  \n\nCURSO \nMATRÍCULAS \n\n2010 \nMATRÍCULAS \n\n2023 \nPART.% \n\n2023 \nEVOL. \n\nANUAL \nSOLDADOR AO ARCO ELÉTRICO E \nOXIGÁS \n\n0 2.401 37,05% - \n\nSOLDADOR AO ARCO ELÉTRICO - \nMAG \n\n0 797 12,30% - \n\nSOLDADOR AO ARCO ELÉTRICO - \nELETRODO REVESTIDO \n\n0 720 11,11% - \n\nSOLDAGEM ELETRODO REVESTIDO \nEM AÇO CARBONO \n\n251 387 5,97% 3,14% \n\nSOLDADOR AO ARCO ELÉTRICO - \nTIG \n\n0 350 5,40% - \n\nSOLDAGEM AO ARCO ELÉTRICO \nTIG NAS POSIÇÕES PLANA, \nHORIZONTAL E VERTICAL \n\n0 197 3,04% - \n\nSOLDAGEM AO ARCO ELÉTRICO \nMAG NAS POSIÇÕES PLANA, \nHORIZONTAL E VERTICAL \n\n0 189 2,92% - \n\nINSPEÇÃO DE QUALIDADE EM \nSOLDAGEM \n\n0 179 2,76% - \n\nSOLDAGEM AO ARCO ELÉTRICO \nELETRODO REVESTIDO NA \nPOSIÇÃO PLANA \n\n0 112 1,73% - \n\nSOLDAGEM ELETRODO REVESTIDO \n1F E 1G \n\n25 85 1,31% 9,13% \n\nSOLDAGEM MAG EM AÇO \nCARBONO \n\n345 83 1,28% -9,68% \n\n\n\n \n \n\n  51 \n\nCURSO \nMATRÍCULAS \n

### Calling GPT Engine for Summarisation

In [24]:
system_prompt = f""""Você deve atuar como um chatbot especializado em fornecer respostas baseadas em um banco de dados de avaliações de estudos de mercado, armazenadas na Solução de Pesquisa de IA do Azure. Suas respostas devem ser exclusivamente baseadas no conteúdo do banco de dados fornecido, sem incluir informações adicionais que não estejam no contexto. Caso não haja uma resposta disponível para a consulta, informe educadamente ao usuário.

O contexto será fornecido como uma lista de objetos, cada um representando uma avaliação de estudo de mercado. Cada objeto contém as seguintes informações:

"chunk": "Conteúdo da revisão do estudo."
"score": "Pontuação de relevância da revisão."

A lista contém as três melhores correspondências com base na similaridade de cosseno entre os embeddings da consulta do usuário e as descrições das avaliações.

Por favor, forneça respostas de forma profissional e natural, sem que o usuário perceba que o sistema está utilizando uma abordagem de recuperação automática de informações. O objetivo é manter uma conversa fluida, como se estivesse interagindo com um especialista humano."""

user_prompt = f""" the user query is: {user_query}
the context is : {context}"""

chat_completions_response = azure_openai_client.chat.completions.create(
    model = os.getenv("GPT_ENGINE"),
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ],
    temperature=0.7
)

print(chat_completions_response.choices[0].message.content)



O curso técnico de soldagem oferece diversas oportunidades de aprendizado e desenvolvimento na área de Metalmecânica. De acordo com os dados disponíveis, podemos observar as seguintes informações sobre as matrículas nos cursos relacionados à soldagem:

- Em 2023, os cursos com maior número de matrículas foram:
  - Soldador ao Arco Elétrico e Oxigás: 2.401 matrículas, representando 37,05% do total.
  - Soldador ao Arco Elétrico - MAG: 797 matrículas, representando 12,30% do total.
  - Soldador ao Arco Elétrico - Eletrodo Revestido: 720 matrículas, representando 11,11% do total.
  - Soldagem Eletrodo Revestido em Aço Carbono: 387 matrículas, representando 5,97% do total.

Além disso, é importante destacar que o curso de Soldagem Eletrodo Revestido em Aço Carbono apresentou um crescimento de 3,14% ao ano. No entanto, é válido ressaltar que essas informações se referem à média anual entre 2018 e 2023.

No que se refere à participação das unidades SENAI nas matrículas em cursos de Formação 